In [14]:
from matplotlib import pyplot as plt
import os
import numpy as np
import pandas as pd
from matplotlib_venn import venn2, venn3, venn3_circles
import scipy

from datetime import timedelta as dt
pd.set_option("display.max_rows",None)
#os.chdir('C:\\Users\\somd7w\\Desktop\\DL_Projects\\preproc_cntr')
cwd = os.getcwd()
data = cwd + '/data'
output = cwd+ '/output'

In [15]:
# Define Variables
pats_imaging = 8817

pats_wWeight = 7923

pats_problems = 6642
pats_cancers = 724
pats_autoimmune=58
pats_congenital_disorders=267
pats_malabsorptive = 2
pats_neuromuscular=469
pats_endocrine = 1159
pats_transplant = 28
pats_unknownproblems = 3935

pats_meds=2624
pats_steroids=2385
pats_throid=559
pats_non_parenteral_nutrition=380
pats_antineoplastics=3590

pats_LDA = 7732
pats_miscLDA = 1633
pats_enteric_tubes = 1052

pats_surgeries = 4762
pats_surghistory=4952

In [17]:
# Import Final Patients List
infile  = output+'/finaldata_withbmionly.csv'
df_M = pd.read_csv(infile, index_col=False)
df_M['CT_SCAN_DATE'] = pd.to_datetime(df_M.CT_SCAN_DATE)
df_M = df_M.loc[:, ~df_M.columns.str.contains('^Unnamed')]
#print('Columns of df_M: ', list(df_M))
print('Length of df_M: ', len(df_M))

df_N = df_M[(df_M['BMI_FOR_AGE_PERCENTILE'] >= 5) & (df_M['BMI_FOR_AGE_PERCENTILE'] <= 95)]
normal_pats = len(df_N)

print('Length of df_N: ', len(df_N))

Length of df_M:  3450
Length of df_N:  2399


In [19]:
df_N_age = df_N[df_N['Age'] >= 3]
print(len(df_N))
print(len(df_N_age))
display(df_N.head(5))

2399
2314


,GIVEN_MRN,NEW_MRN,PAT_ID,CT_SCAN_DATE,ACC,STUDYRESULT,IMPRESSION,WEIGHT,HEIGHT,BMI_FOR_AGE_PERCENTILE,...,LDA_GROUP,LDA_ID,LDAS,SURGERY,SURGERY_to_Imaging_days,ValidPatient,Age,Sex,Race,AgeGroup
0,1343508,NaN,Z704566,2014-06-12 16:40:00,8018969,CLINICAL HISTORY: SBO. Pt has had severe abd ...,IMPRESSION: Findings concerning for a small bo...,23.15,125.0,72.284778,...,unknown_ldas,NaN,NaN,surgery,> 30,1,7.429290,Male,White,7.0
1,11699446,NaN,Z1931299,2017-11-05 00:38:00,8842054,CLINICAL HISTORY: 2 yr old male with history o...,1. No acute traumatic abnormality in the abdo...,10.80,84.0,84.492605,...,unknown_ldas,NaN,NaN,no_surgery,NaN,1,2.137059,Male,Black or African American,2.0
2,1299521,NaN,Z664930,2014-06-23 01:06:00,8024856,CLINICAL HISTORY: [8-year-old with abdominal p...,IMPRESSION: Grade 3 splenic laceration with as...,23.70,118.0,72.637945,...,unknown_ldas,NaN,NaN,no_surgery,NaN,1,8.260371,Male,Black or African American,8.0
4,11437502,NaN,Z1465933,2014-05-14 23:49:00,8000527,CLINICAL HISTORY: 9 yo F with blunt trauma to ...,IMPRESSION: 1. Left grade V renal injury with...,30.80,124.0,87.314049,...,unknown_ldas,NaN,NaN,no_surgery,NaN,1,9.627373,Female,White,10.0
8,11063680,NaN,Z1091411,2014-05-24 02:02:00,8006713,"CLINICAL HISTORY: 5-year-old, s/p MVC with ele...",IMPRESSION: No traumatic injury of the abdomen...,20.75,111.0,84.833169,...,unknown_ldas,NaN,NaN,surgery,> 30,1,5.197492,Male,White,5.0


In [20]:
df_N.to_csv(output+'/patlist_with_validBMI.csv')
df_N_age.to_csv(output+'/patlist_with_validBMI_age3nabove.csv')

In [8]:
infile  = output+'/predictions_v1.csv'
df_P = pd.read_csv(infile, index_col=False)
print('Columns of df_P: ', list(df_P))
print('Length of df_P: ', len(df_P))
print('# of Unique patients: ', len(df_P.subject_id.unique()))

Columns of df_P:  ['subject_id', 'area_mm2', 'sagittal_series', 'axial_series']
Length of df_P:  7158
# of Unique patients:  4969


In [21]:
# Filter df_M to match the unique patients in df_P
unique_pats = df_P.subject_id.unique()
df_missing = df_N[~df_N['PAT_ID'].isin(unique_pats)]
print('Length of df_missing:', len(df_missing))

df_missing.to_csv(output+'/missingCT.csv')

df_N = df_N[df_N['PAT_ID'].isin(unique_pats)]
print('Length of df_N:', len(df_N))

Length of df_missing: 409
Length of df_N: 1990


In [293]:
# Handle patients with more than one image series and hence more than one predicted muscle area.
df_P = df_P.sort_values(by=['subject_id'])

#Drop patients with multiple predictions but same area
df_P = df_P.drop_duplicates(['subject_id','area_mm2'], keep='last')
print('Length of df_P after dropping equal area duplicates: ', len(df_P))

df_P_duplicates = df_P[df_P.duplicated('subject_id',keep=False)==True]
print('Length of df_P_duplicates:', len(df_P_duplicates))

duplicate_pats = df_P_duplicates.subject_id.unique()
print('# unique pats with dupliates: ', len(df_P_duplicates.subject_id.unique()))

df_P_unique =  df_P.drop_duplicates(['subject_id'], keep='last')

Length of df_P after dropping equal area duplicates:  5831
Length of df_P_duplicates: 1355
# unique pats with dupliates:  493


In [294]:
# Stats:
print("Total number of area predictions: ", len(df_P))
print("Number of unique patients with area prediction: ", len(unique_pats))
print("Number of patients with multiple area predictions: ", len(duplicate_pats))

Total number of area predictions:  5831
Number of unique patients with area prediction:  4969
Number of patients with multiple area predictions:  493


# ANALYSIS

In [298]:
# no duplicate dataframe
df_P_noduplicates = df_P[~df_P['subject_id'].isin(duplicate_pats)]
print(len(df_P_noduplicates))

df_P_noduplicates.rename(columns={'subject_id': 'PAT_ID'}, inplace=True)
print(list(df_P_noduplicates))


# no duplicate dataframe
df_P_unique.rename(columns={'subject_id': 'PAT_ID'}, inplace=True)
print(list(df_P_unique))

4476
['PAT_ID', 'area_mm2', 'sagittal_series', 'axial_series']
['PAT_ID', 'area_mm2', 'sagittal_series', 'axial_series']


/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [203]:
df_PN = pd.merge(df_P_noduplicates, df_N, on='PAT_ID', how='inner')
df_PNunique = pd.merge(df_P_unique, df_N, on='PAT_ID', how='inner')

print("Number of normal unique patients with studies: ", len(df_PNunique))
print("Number of normal patients without duplicate studies: ", len(df_PN))
print("Total number of normal patients in dataset: ", normal_pats)
print("Number of normal patients with studies missing: ", normal_pats - len(df_N))

Number of normal unique patients with studies:  1990
Number of normal patients without duplicate studies:  1772
Total number of normal patients in dataset:  2399
Number of normal patients with studies missing:  409
